In [1]:
import requests
import json
import random
import time

# Base URL of the FastAPI server
BASE_URL = "http://localhost:8000"

# Optional: user_id for personalized /locations/recommended
# Get this from /auth/login if you want to test recommendations
USER_ID = None  # e.g. "6e0e5c7e-1234-4567-890a-bcdef1234567"

HEADERS = {"X-User-ID": USER_ID} if USER_ID else {}

print("BASE_URL =", BASE_URL)
print("USER_ID =", USER_ID)


BASE_URL = http://localhost:8000
USER_ID = None


## 🗑️ Delete All Existing Locations (Optional)

⚠️ **Warning:** This will permanently delete all locations from the database. Run this cell only if you want to start fresh.

In [ ]:
# Set to True to enable deletion
DELETE_ALL = False

if DELETE_ALL:
    # Fetch all existing locations
    url = f"{BASE_URL}/locations/"
    res = requests.get(url)
    
    if res.status_code == 200:
        all_locations = res.json()
        deleted_count = 0
        
        print(f"Found {len(all_locations)} locations. Deleting...")
        
        for location in all_locations:
            location_id = location.get("id")
            if location_id:
                delete_url = f"{BASE_URL}/locations/{location_id}"
                delete_res = requests.delete(delete_url)
                
                if delete_res.status_code == 200:
                    deleted_count += 1
                    print(f"✓ Deleted: {location.get('name', 'Unknown')} ({location_id})")
                else:
                    print(f"✗ Failed to delete: {location.get('name', 'Unknown')} - Status {delete_res.status_code}")
        
        print(f"\n✅ Deleted {deleted_count} out of {len(all_locations)} locations")
    else:
        print(f"Failed to fetch locations. Status: {res.status_code}")
else:
    print("⚠️ Deletion is disabled. Set DELETE_ALL = True to enable.")

In [ ]:
location_names = [
    "Victoria Peak", "Lion Rock", "Dragon's Back", "Mount Parker", "Lantau Peak",
    "Tai Mo Shan", "Sharp Peak", "Sunset Peak", "Needle Hill", "Ma On Shan",
    "High West", "Pat Sin Leng", "Jardine's Lookout", "The Twins", "Castle Peak",
    "Pineapple Mountain", "Shing Mun Reservoir", "Aberdeen Peak", "Clear Water Bay Peak",
    "Ap Lei Chau Trail", "Cape D’Aguilar", "Sai Wan Peak"
]

areas = ["Central", "Kowloon", "New Territories", "Lantau", "Hong Kong Island"]
price_levels = [1, 2, 3, 4]

bulk_created_ids = []

for name in location_names:
    payload = {
        "name": name,
        "description": f"A scenic location known as {name}.",
        "maps_url": "https://maps.example.com",
        "price_level": random.choice(price_levels),
        "area": random.choice(areas),
    }

    url = f"{BASE_URL}/locations/"
    try:
        res = requests.post(url, json=payload)
        print(f"Created {name} → Status {res.status_code}")
        data = res.json()
        loc_id = data.get("id")
        if loc_id:
            bulk_created_ids.append(loc_id)
    except Exception as e:
        print(f"Error creating {name}: {e}")

time.sleep(0.5)
print("\nTotal created:", len(bulk_created_ids))
bulk_created_ids[:5]


Created Victoria Peak → Status 200
Created Lion Rock → Status 200
Created Dragon's Back → Status 200
Created Mount Parker → Status 200
Created Lantau Peak → Status 200
Created Tai Mo Shan → Status 200
Created Sharp Peak → Status 200
Created Sunset Peak → Status 200
Created Needle Hill → Status 200
Created Ma On Shan → Status 200
Created High West → Status 200
Created Pat Sin Leng → Status 200
Created Jardine's Lookout → Status 200
Created The Twins → Status 200
Created Castle Peak → Status 200
Created Pineapple Mountain → Status 200
Created Shing Mun Reservoir → Status 200
Created Aberdeen Peak → Status 200
Created Clear Water Bay Peak → Status 200
Created Ap Lei Chau Trail → Status 200
Created Cape D’Aguilar → Status 200
Created Sai Wan Peak → Status 200
Created Hawk's Nest → Status 200
Created Tung Chung Valley → Status 200

Total created: 24


['7cf922ab-8e93-4d6e-b7dd-686a0c2f2e9d',
 'd535d0c4-8384-44f9-9e9f-4792b7c0c859',
 '05e2f8fe-9f79-48ea-bce7-3e1dd7b5b682',
 'ac9dc280-da80-4202-837d-0e0508312680',
 '0912babd-ccd6-4f11-9771-8dcace4a0fcc']

In [2]:
url = f"{BASE_URL}/locations/"
res = requests.get(url)

print("Status:", res.status_code)
all_locations = res.json()
print("Total locations returned:", len(all_locations))
print(json.dumps(all_locations[:3], indent=2))  # show first 3


Status: 200
Total locations returned: 52
[
  {
    "name": "Victoria Peak",
    "description": "The highest hill on Hong Kong Island with panoramic skyline views.",
    "maps_url": "https://maps.app.goo.gl/CxbF1j1xo2FqiHzH8",
    "price_level": 2,
    "area": "Hong Kong Island",
    "region": null,
    "summary": null,
    "duration": null,
    "opening_hours": null,
    "id": "de3763f6-41a4-4370-bb03-0baa7f921ecc",
    "created_at": "2025-11-18T19:44:52.369903+08:00"
  },
  {
    "name": "Tsim Sha Tsui Promenade",
    "description": "Iconic waterfront walkway.",
    "maps_url": "https://maps.app.goo.gl/msnmEiW7NiyDLjMu9",
    "price_level": 1,
    "area": "Kowloon",
    "region": null,
    "summary": null,
    "duration": null,
    "opening_hours": null,
    "id": "70389361-b298-4a67-84cc-1a03953b41bc",
    "created_at": "2025-11-18T19:54:39.240075+08:00"
  },
  {
    "name": "Mong Kok Night Market",
    "description": "Bustling night market with food and fashion.",
    "maps_url": "h

In [ ]:
import os
import requests

BASE_URL = "http://127.0.0.1:8000"
IMAGE_FOLDER = "backend/media/location_images"

def normalize(name: str):
    return name.lower().replace(" ", "").replace("'", "")

# 1. Load all images
files = [
    f for f in os.listdir(IMAGE_FOLDER)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
]
print(f"Found {len(files)} images.")

# 2. Fetch all locations
resp = requests.get(f"{BASE_URL}/locations")
locations = resp.json()

loc_map = { normalize(loc["name"]): loc["id"] for loc in locations }

# 3. Attach images via the new API route
added = 0
missed = 0

for img in files:
    base = os.path.splitext(img)[0]   # e.g. lionrock
    if base not in loc_map:
        print(f"⚠️ No matching location for {img}")
        missed += 1
        continue
    
    loc_id = loc_map[base]

    print(f"Linking {img} → {loc_id}")

    payload = {"file_path": img}
    r = requests.post(f"{BASE_URL}/locations/{loc_id}/images/from-path", json=payload)

    if r.status_code == 200:
        added += 1
    else:
        print("❌ Error:", r.text)

print(f"\nDone: {added} added, {missed} unmatched.")


In [4]:
# Example: filter by area and price_level
url = f"{BASE_URL}/locations/?area=Central&price_level=2"
res = requests.get(url)

print("Status:", res.status_code)
filtered_locations = res.json()
print("Filtered locations count:", len(filtered_locations))
print(json.dumps(filtered_locations[:3], indent=2))


Status: 200
Filtered locations count: 0
[]


In [5]:
if bulk_created_ids:
    test_location_id = bulk_created_ids[0]
    print("Using test_location_id from bulk_created_ids:", test_location_id)
elif all_locations:
    test_location_id = all_locations[0]["id"]
    print("No bulk_created_ids, using first from all_locations:", test_location_id)
else:
    test_location_id = None
    print("No locations available to test.")

test_location_id


Using test_location_id from bulk_created_ids: 7cf922ab-8e93-4d6e-b7dd-686a0c2f2e9d


'7cf922ab-8e93-4d6e-b7dd-686a0c2f2e9d'

In [6]:
if not test_location_id:
    raise RuntimeError("No test_location_id available. Run previous cells first.")

url = f"{BASE_URL}/locations/{test_location_id}"
res = requests.get(url)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "location": {
    "name": "Victoria Peak",
    "description": "A scenic location known as Victoria Peak.",
    "maps_url": "https://maps.example.com",
    "price_level": 3,
    "area": "Central",
    "region": null,
    "summary": null,
    "duration": null,
    "opening_hours": null,
    "id": "7cf922ab-8e93-4d6e-b7dd-686a0c2f2e9d",
    "created_at": "2025-11-23T14:32:47.320248+08:00"
  },
  "images": [],
  "tags": []
}


In [7]:
update_payload = {
    "name": "Updated Test Location",
    "price_level": 3,
}

url = f"{BASE_URL}/locations/{test_location_id}"
res = requests.put(url, json=update_payload)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "name": "Updated Test Location",
  "description": "A scenic location known as Victoria Peak.",
  "maps_url": "https://maps.example.com",
  "price_level": 3,
  "area": "Central",
  "region": null,
  "summary": null,
  "duration": null,
  "opening_hours": null,
  "id": "7cf922ab-8e93-4d6e-b7dd-686a0c2f2e9d",
  "created_at": "2025-11-23T14:32:47.320248+08:00"
}


In [8]:
import os

image_path = "test_image.jpg"  # change if needed

if os.path.exists(image_path):
    url = f"{BASE_URL}/locations/{test_location_id}/images"
    with open(image_path, "rb") as f:
        files = {"file": (os.path.basename(image_path), f, "image/jpeg")}
        res = requests.post(url, files=files)

    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
else:
    print(f"Image file not found: {image_path}. Skipping image upload test.")


Image file not found: test_image.jpg. Skipping image upload test.


In [ ]:
tags_payload = {
    "tags": ["hiking", "scenic", "sunset"]
}

url = f"{BASE_URL}/locations/{test_location_id}/tags"
res = requests.post(url, json=tags_payload)

print("Status:", res.status_code)
tags_response = res.json()
print(json.dumps(tags_response, indent=2))

tag_ids = [t.get("id") for t in tags_response.get("added_tags", []) if t.get("id") is not None]
tag_ids


: 

In [ ]:
if tag_ids:
    tag_id = tag_ids[0]
    url = f"{BASE_URL}/locations/{test_location_id}/tags/{tag_id}"
    res = requests.delete(url)

    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
else:
    print("No tag_ids available to delete.")


: 

In [ ]:
if not USER_ID:
    print("USER_ID is not set. Skipping /locations/recommended test.")
else:
    url = f"{BASE_URL}/locations/recommended?limit=10"
    res = requests.get(url, headers=HEADERS)
    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))


: 

In [ ]:
cleanup = False  # set to True if you want to delete the test location

if cleanup and test_location_id:
    url = f"{BASE_URL}/locations/{test_location_id}"
    res = requests.delete(url)
    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
elif not cleanup:
    print("Cleanup is disabled. Set cleanup = True to delete the test location.")
else:
    print("No test_location_id to delete.")


: 